In [1]:
import pyodbc
from bs4 import BeautifulSoup
from urllib.parse import urljoin,urlparse
import requests
import re

In [2]:
conn=pyodbc.connect('Driver={SQL Server};''Server=DESKTOP-CDBHGSC;'
                    'Database=HousePriceMorocco;''Trusted_connection=yes')

In [3]:
if conn:
    print('True')

True


In [4]:
cursor = conn.cursor()

In [5]:
url = 'https://www.masaken.ma/fr/' 
response = requests.get(url)
response

<Response [200]>

In [7]:
query = """
INSERT INTO Web (id_site, nom_site, url_site)
VALUES (?, ?, ?)
"""
cursor.execute(query, 10, "masaken", url)

In [8]:
soup = BeautifulSoup(response.content, 'html.parser')
num = 0
nb = 2024

div_element1 = soup.find_all('div', class_='compact-links')
for div in div_element1:
    h3_child = div.find('h3')
    if h3_child and (h3_child.text == "Vente appartement" or h3_child.text == "Vente maison"):
        a_tags = div.find_all('a')
        for a in a_tags:
            num += 10
            relative_link = a.get('href')
            if relative_link:
                absolute_link = urljoin(url, relative_link)
                parsed_url = urlparse(absolute_link)
                path_segments = parsed_url.path.split('/')
                city = path_segments[-1]
                response = requests.get(absolute_link)
                soup = BeautifulSoup(response.content, 'html.parser')

                div_element = soup.find_all('div', class_='annBlocDesc')
                for dv in div_element:
                    num += 1
                    annBloc = dv.find('div')
                    prix = annBloc.find('div', class_='annPrice')
                    p = prix.find('span', dir="ltr")
                    price_str = p.text.strip()
                    price_str = re.sub(r'[^\d.,]', '', price_str) 
                    price_str = price_str.replace(' ', '') 
                    price = float(price_str)
                    nom = annBloc.find("h2")
                    match = re.search(r'(\d+)\s*pièces?.*?(\d+)\s*m²', nom.text)
                    
                    if match:
                        pieces = int(match.group(1))
                        metrage = int(match.group(2))

                    quartier = annBloc.find("h3")
                    text = quartier.text
                    start = text.find('«') + 1
                    end = text.find('»')
                    
                    substring = text[start:end].strip()
                    
                    query = """
                    SELECT id_location
                    FROM Localisation 
                    WHERE city = ? AND district = ? 
                    """
                    cursor.execute(query, city, substring)
                    result = cursor.fetchone()
                    
                    if result:
                        id_location = result[0]
                    else:
                        insert_location_query = """
                        INSERT INTO Localisation (id_location,city, district) 
                        VALUES (?,?, ?)
                        """
                        cursor.execute(insert_location_query,num, city, substring)
                        id_localisation = num
                    nb=nb+1
                    insert_maison_query = """
                    INSERT INTO House (id_house, price, number_of_rooms, area, id_site, id_location) 
                    VALUES (?, ?, ?, ?, ?, ?)
                    """
                    cursor.execute(insert_maison_query, nb, price, pieces, metrage, 10, id_localisation)


In [9]:
conn.commit()

In [10]:
cursor.close()
conn.close()